# Algoritmo de Naive Bayes
Implementação e comentários

### Implementação

#### Etapas



##### Treino
-   calcular média, variância e frequencia (prior) para cada classe

##### Predição
-   calcular a probabilidade posterior pra cada classe a distribuição Gaussiana e a função alvo
$$y = argmax_y \log(P(x_1|y))+ \log(P(x_3|y))+...+ \log(P(x_n|y))+ \log(P(y))$$
-   escolhe a classe com maior probabilidade posterior

#### Código

In [2]:
from sklearn.model_selection import train_test_split
from sklearn import datasets
import numpy as np

def accuracy(y_true, y_pred):
    accuracy = np.sum(y_true == y_pred) / len(y_true)
    return accuracy


In [11]:
class NaiveBayes:

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self._classes = np.unique(y)
        n_classes = len(self._classes)

        self._mean = np.zeros((n_classes, n_features), dtype=np.float64)
        self._var = np.zeros((n_classes, n_features), dtype=np.float64)
        self._priors = np.zeros(n_classes, dtype=np.float64)

        for idx, c in enumerate(self._classes):
            X_c = X[y==c]
            self._mean[idx, :] = X_c.mean(axis=0)
            self._var[idx, :] = X_c.var(axis=0)
            self._priors[idx] = X_c.shape[0] / float(n_samples)


    def predict(self, X):
        y_pred = [self._predict(x) for x in X]
        return np.array(y_pred)
    
    def _predict(self, x):
        posteriors = []

        for idx, c in enumerate(self._classes):
            prior = np.log(self._priors[idx])
            posterior = np.sum(np.log(self._pdf(idx, x)))
            posterior = posterior + prior
            posteriors.append(posterior)
        
        return  self._classes[np.argmax(posteriors)]

    def _pdf(self, idx, x):
        mean = self._mean[idx]
        var = self._var[idx]
        numerator = np.exp(-((x - mean) ** 2) / (2 * var))
        denominator = np.sqrt(2 * np.pi * var)

        return numerator / denominator

In [5]:
X, y = datasets.make_classification(
    n_samples=1000, n_features=10, n_classes=2, random_state=123
)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=123
)

In [12]:
nb = NaiveBayes()
nb.fit(X_train, y_train)
prediction = nb.predict(X_test)
acc = accuracy(y_test, prediction)
print(f"Accuracy: {acc}")

Accuracy: 0.965


### Base teórica matemática

Probabilidade conjunta: é a probabilidade de 2 eventos A e B ocorrerem
- $P(A)$: Probabilidade de ocorrer o evento A
- $P(B)$: Probabilidade de ocorrer o evento B
- $P(A, B)$: Probabilidade de A e B ocorrerem
- $P(A, B) = P(A)\times P(B)$: Probabilidade de A e B ocorrerem
- $P(A|B)$: Probabilidade do evento A acontecer dado o evento B ocorrido

O Teorema de Bayes nos diz que
$$P(A|B) = \frac{P(B|A)\times P(A)}{P(B)}$$

Dessa forma, seja 
- $X = (x_1, x_2,..., x_n)$ nosso vetor de features
- $y$ nosso _target_

Podemos traduzir o teorema para
$$P(y|X) = \frac{P(X|y)\times P(y)}{P(X)} = \frac{P(x_1|y)\times P(x_2|y) \times ...\times P(x_n|y)\times P(y)}{P(X)}$$

Quando temos um conjunto de classes $Y_i$ onde $i=1,2,...,k$, então podemos dizer que
$$P(Y_i|X) = \frac{P(x_1|Y_i)\times P(x_2|Y_i) \times ...\times P(x_n|Y_i)\times P(Y_i)}{P(X)}, i=1,2,3,...,k$$
Portanto nossa classificação pode ser feita através de qual classe $y \in Y$ nosso evento X pertece através da maior probabilidade de ocorrência
$$y = argmax_yP(y|X) = \frac{P(x_1|y)\times P(x_2|y) \times ...\times P(x_n|y)\times P(y)}{P(X)}$$


Podemos dizer que
$$y = argmax_y P(x_1|y)\times P(x_2|y)\times...\times P(x_n|y)\times P(y)$$
$$y = argmax_y \log(P(x_1|y))+ \log(P(x_3|y))+...+ \log(P(x_n|y))+ \log(P(y)) $$

Também são úteis as seguintes definições 
- $P(y)$: Probabilidade a priori
- $P(x_i|y)$: Probabilidade condicional de ser a classe
- $\mu$: média da classe y
- $\sigma^2$: variância da classe y

O cálculo $P(x_i|y)$ será feito com a distribuição Gaussiana, oque nos leva a
$$P(x_i|y) = \frac{1}{\sqrt{2\pi\sigma^2_y}}\times exp(-\frac{(x_i-\mu_y)^2}{2\sigma^2_y})$$